In [1]:
import os
import time
import glob
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns




In [2]:
os.listdir('/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/') # categories of summaries

['politics', 'sport', 'tech', 'entertainment', 'business']

In [3]:
# Now we define paths for articles, summaries and maintain a list of categories of our data
article_path = '/kaggle/input/bbc-news-summary/BBC News Summary/News Articles'
summary_path = '/kaggle/input/bbc-news-summary/BBC News Summary/Summaries/'
categories = ['politics', 'sport', 'tech', 'entertainment', 'business']